In [1]:
import bs4
import requests
import pandas as pd
import numpy as np
import time
import os
from IPython.display import clear_output

In [2]:
# 요청 함수
def getSource(site) :
    # 해더 정보 셋팅
    header_info = {
        'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    # 요청한다.
    response = requests.get(site, headers=header_info)
    return response.text

In [3]:
# 한 페이지의 데이터를 수집해 저장하는 함수
def getData(source):
    
    # bs4 객체를 생성한다.
    soup = bs4.BeautifulSoup(source)
    
    # 영화 전체를 가져온다.
    li_list = soup.select('#mainContent > div > div.box_ranking > ol > li')
    
    # 데이터를 담을 딕셔너리
    data_dict = {
        '영화제목' : [],
        '평점' : [],
        '예매율' : [],
        '개봉일' : [],
        '영화설명' : [],
        '관람등급' : []
    }
    
    for li_tag in li_list :
        # print(li_tag)
        # print('---------------------------')
        
        # 영화제목
        a1 = li_tag.select_one('a.link_txt')
        data1 = getText(a1)
        
        # 평점
        a2 = li_tag.select_one('span.txt_grade')
        data2 = getText(a2)
        
        # 예매율
        a3 = li_tag.select_one('div > div.thumb_cont > span.txt_append > span:nth-child(2) > span')
        data3 = getText(a3)[:-1]
         
        # 개봉일
        a4 = li_tag.select_one('div > div.thumb_cont > span.txt_info > span')
        data4 = getText(a4)
        
        # 영화 설명
        a5 = li_tag.select_one(' div > div.thumb_item > div.poster_info > a')
        data5 = getText(a5)
        
        # 관람등급
        a6 = li_tag.select_one('div > div.thumb_item > div.poster_movie > span.txt_tag > span')
        data6 = getText(a6)
        
        # 포스터
        a7 = li_tag.select_one('div > div.thumb_item > div.poster_movie > img')
        src = a7.attrs['src']
        data7 = a7['src']
        
        # 이미지 파일 데이터를 받아온다.
        img_res = requests.get(src)
        
        # 파일 이름 정제
        file_name = data1
        file_name = file_name.replace('\\', ' ')
        file_name = file_name.replace('/', ' ')
        file_name = file_name.replace(':', ' ')
        file_name = file_name.replace('?', ' ')
        file_name = file_name.replace('"', ' ')
        file_name = file_name.replace('*', ' ')
        file_name = file_name.replace('<', ' ')
        file_name = file_name.replace('>', ' ')
        file_name = file_name.replace('|', ' ')
        
        # 이미지를 저장한다.
        with open(f'./03_poster/{file_name}.jpg', 'wb') as fp :
            fp.write(img_res.content)
        
        # 데이터를 담는다
        data_dict['영화제목'].append(data1)
        data_dict['평점'].append(data2)
        data_dict['예매율'].append(data3)
        data_dict['개봉일'].append(data4)
        data_dict['영화설명'].append(data5)
        data_dict['관람등급'].append(data6)
        
    # 데이터 프레임 생성
    df1 = pd.DataFrame(data_dict)
    df1.to_csv('./03_data.csv', encoding='utf-8-sig', index=False)

In [4]:
# 전달된 태그로 부터 문자열 데이터를 추출해 반환하는 함수
def getText(tag):
    # 태그가 없다면
    if tag == None :
        return np.nan
    # 문자열을 가져온다.
    data = tag.text.strip()
    # 길이가 0 이라면...
    if len(data) == 0 :
        return np.nan
    return data

In [5]:
# 페이지의 주소
site = 'https://movie.daum.net/ranking/reservation'

source = getSource(site)

getData(source)

print('수집완료')

수집완료
